In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import datetime
from datetime import date, timedelta
from sklearn.cluster import KMeans
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import plotly.offline as py
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from keras.models import Sequential
from keras.layers import LSTM,Dense
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
data = pd.read_csv("../input/coronavirus-2019ncov/covid-19-all.csv")

In [ ]:
data.head()

In [ ]:
data['Country/Region'].unique()

In [ ]:
data = data[data['Country/Region'] ==  'Jordan']

In [ ]:
data.head()

In [ ]:
cnfrm = data.Confirmed.max()
cnfrm

In [ ]:
dth =  data.Deaths.max()
dth

In [ ]:
rcv = data.Recovered.max()
rcv

In [ ]:
data = data.tail(21)

In [ ]:
plt.figure(figsize=(50,40))
plt.bar(data.Date, data.Confirmed,label="Confirm")
plt.bar(data.Date, data.Recovered,label="Recovery")
plt.bar(data.Date, data.Deaths,label="Death")
plt.xlabel('Date')
plt.ylabel("Count")
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.legend(frameon=True, fontsize=42)
plt.title('Confrim vs Recovery vs Death',fontsize=30)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(30,10))
ax=sns.scatterplot(x="Date", y="Confirmed", data=data,
             color="black",label = "Confirm")
ax=sns.scatterplot(x="Date", y="Recovered", data=data,
             color="red",label = "Recovery")
ax=sns.scatterplot(x="Date", y="Deaths", data=data,
             color="blue",label = "Death")
plt.xticks(fontsize=10)
plt.yticks(fontsize=20)

plt.legend(frameon=True, fontsize=22)
plt.plot(data.Date,data.Confirmed,zorder=1,color="black")
plt.plot(data.Date,data.Recovered,zorder=1,color="red")
plt.plot(data.Date,data.Deaths,zorder=1,color="blue")

In [ ]:
import folium
map = folium.Map(location=[30.5852,36.2384 ], zoom_start=6,tiles='Stamen Toner')

for lat, lon,Confirmed,Recovered,Deaths in zip(data['Latitude'], data['Longitude'],data['Confirmed'],data['Recovered'],data['Deaths']):
    folium.CircleMarker([lat, lon],
                        radius=5,
                        color='red',
                      popup =(
                    'Confirmed: ' + str(Confirmed) + '<br>'
                      'Recovered: ' + str(Recovered) + '<br>'
                      'Deaths: ' + str(Deaths) + '<br>'),

                        fill_color='red',
                        fill_opacity=0.7 ).add_to(map)
map

In [ ]:
pr_data_r = data.loc[:,['Date','Confirmed']]
pr_data_r.columns = ['ds','y']
pr_data_r.head()

In [ ]:
m=Prophet()
m.fit(pr_data_r)
future=m.make_future_dataframe(periods=60)
forecast_r=m.predict(future)
forecast_r

In [ ]:
rec = forecast_r.loc[:,['ds','trend']]
rec = rec[rec['trend']>0]
rec=rec.tail(14)
rec.columns = ['Date','Confirmed']
rec.head()

In [ ]:
fig_r = plot_plotly(m, forecast_r)
py.iplot(fig_r) 

fig_r = m.plot(forecast_r,xlabel='Date',ylabel='Confirmation Count')